In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from analyzer import *
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [13]:
labels = ["l1", "l2"]
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [14]:
analyzer = Analyzer(root_dir="..")

In [22]:
analyzer.model_overview(y_test, y_pred, labels, "1.0", plot_metrics=False, overwrite=True, metrics_size=(5, 5))

In [17]:
analyzer.compare_models("1.0", "1.1", overwrite=True)